## Binary questions: Feature Selection

The questions posed to GPT were based on a true-false basis. To evaluate which questions are most relevant to explain the ratings, we are most interested in performing the Chi-square test. 

In [5]:
%matplotlib inline
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
#import json
#import scipy.stats as stats
#import requests 

### Extracting a Dataset containing all questions and splitting into train and test set

In [73]:
f = open('Data/trope_questions.txt', 'r')

line = f.readline()
questions = []

while len(line)!=0:
    questions.append(line.strip())
    line = f.readline()
    
questions = questions[1:-1]

In [74]:
questions

['- Is there a wedding stopped at the altar trope ?',
 '- Is there a best friend to lovers trope ?',
 '- Is there a enemies to lovers trope ?',
 '- Is the romance impossible because of the different social status of the protagonists ?',
 '- Has one of the lovers a serious illness ?',
 '- Is this love at first sight?',
 '- Is there a meet-cute trope ?',
 '- is there a break up ?',
 '- Is there a one night stand ?',
 '- Is one of the main characters initially involved into a different relationship?',
 '- Is there a love triangle ?',
 '- Is the ending sad ?',
 '- Is there a LGBT couple ?',
 '- Is there infidelity ?',
 '- Do they start dating as a bet ?',
 '- Is there a fake dating trope ?',
 '- Is the male protagonist a bad boy ?',
 "- Is the movie linked to a special holliday (christmas, valentine's day...)?",
 '- Is there an empowered woman having regrets ?',
 '- Is there a reunion after a long time spent apart ?']

In [77]:
questions_dict = {'q_'+ str((i+1)) : questions[i] for i in range(len(questions))}

In [58]:
characters = pd.read_csv('./Data/Preprocessed/preprocessed_characters.tsv', delimiter='\t')
movies_augmented = pd.read_csv('./Data/Preprocessed/movie.metadata.augmented.tsv', delimiter='\t')
romances_with_binary = pd.read_csv('./Data/Preprocessed/romances.with.binary.tsv', delimiter='\t')

In [59]:
romances_with_binary.columns

Index(['index', 'movie_id', 'movie_name', 'revenue', 'runtime', 'languages',
       'countries', 'movie_release', 'genres', 'imdb_id', 'rating', 'nb_votes',
       'binary_answers'],
      dtype='object')

In [60]:
gpt_answers = romances_with_binary[['movie_id', 'binary_answers']].copy()
gpt_answers['response_length'] = gpt_answers['binary_answers'].apply(lambda ans: len(ans))
gpt_answers.head(3)

,movie_id,binary_answers,response_length
0,6631279,01001000000001111000,20
1,21926710,00000001000001000100,20
2,26067101,01000000001000001000,20


In [61]:
def print_length_of_string_occurrences(df, print_incorrect_rows=False):
    for length in df['response_length'].unique():
        print(f'{len(df[df.response_length==length])} row(s) has/have length {length}')
        if print_incorrect_rows and length != 20:
            print(df[df.response_length==length])



In [63]:
print_length_of_string_occurrences(gpt_answers, print_incorrect_rows=False)

5811 row(s) has/have length 20
70 row(s) has/have length 21
1 row(s) has/have length 19


In [57]:
for i in range(20):
    # print(i)
    romances_with_binary['binary_answers'].apply(lambda x: print(len(x)))
    #loaded_romances_with_binary["Q"+str(i + 1)] = loaded_romances_with_binary['binary_answers'].apply(lambda x: x[i])

20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
21
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
21
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
21
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
21
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
21
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
2

In [69]:

bits = {'0', '1', '2'}
gpt_answers['binary_answers'] = gpt_answers['binary_answers'].apply(lambda b: ''.join(c for c in b if c in bits))

#recalculate the response length
gpt_answers['response_length'] = gpt_answers['binary_answers'].apply(lambda ans: len(ans))



In [70]:
print_length_of_string_occurrences(gpt_answers, print_incorrect_rows=True)

5881 row(s) has/have length 20
1 row(s) has/have length 19
      movie_id       binary_answers  response_length
1569   2210682  0000000000111100010               19


The columns with incorrect lengths are now replaced with a string of '2's (20 concatenated two's). 

In [71]:
gpt_answers['binary_answers'] = gpt_answers['binary_answers'].apply(lambda b: b if len(b)==20 else '2'*20)

In [72]:
gpt_answers.drop(columns=['response_length'], inplace=True)

Now, after this preprocessing, let's separate the answers to each question in a separate column of the dataframe.


In [78]:
for q_nb, question in enumerate(questions):
    gpt_answers[f'q_{q_nb}'] = gpt_answers['binary_answers'].apply(lambda b: int(b[q_nb]))
    
gpt_answers.head(5)

,movie_id,binary_answers,q_0,q_1,q_2,q_3,q_4,q_5,q_6,q_7,...,q_10,q_11,q_12,q_13,q_14,q_15,q_16,q_17,q_18,q_19
0,6631279,01001000000001111000,0,1,0,0,1,0,0,0,...,0,0,0,1,1,1,1,0,0,0
1,21926710,00000001000001000100,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
2,26067101,01000000001000001000,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,12053509,01000000000111011000,0,1,0,0,0,0,0,0,...,0,1,1,1,0,1,1,0,0,0
4,7028314,00010011010001001000,0,0,0,1,0,0,1,1,...,0,0,0,1,0,0,1,0,0,0


In [79]:
# we remove all the rows where the answer is '2'*20
relevant_answers = gpt_answers[gpt_answers['binary_answers']!='2'*20].copy()

In [80]:
movies_augmented.columns

Index(['movie_id', 'movie_name', 'revenue', 'runtime', 'languages',
       'countries', 'movie_release', 'genres', 'imdb_id', 'rating',
       'nb_votes'],
      dtype='object')

In [81]:
relevant_answers.columns

Index(['movie_id', 'binary_answers', 'q_0', 'q_1', 'q_2', 'q_3', 'q_4', 'q_5',
       'q_6', 'q_7', 'q_8', 'q_9', 'q_10', 'q_11', 'q_12', 'q_13', 'q_14',
       'q_15', 'q_16', 'q_17', 'q_18', 'q_19'],
      dtype='object')

In [84]:
relevant_answers.merge(movies_augmented[['movie_id','rating']], on='movie_id', how='inner')

,movie_id,binary_answers,q_0,q_1,q_2,q_3,q_4,q_5,q_6,q_7,...,q_11,q_12,q_13,q_14,q_15,q_16,q_17,q_18,q_19,rating
0,6631279,01001000000001111000,0,1,0,0,1,0,0,0,...,0,0,1,1,1,1,0,0,0,5.8
1,21926710,00000001000001000100,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,6.1
2,26067101,01000000001000001000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,6.4
3,12053509,01000000000111011000,0,1,0,0,0,0,0,0,...,1,1,1,0,1,1,0,0,0,6.0
4,7028314,00010011010001001000,0,0,0,1,0,0,1,1,...,0,0,1,0,0,1,0,0,0,5.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5768,4037444,00010100010001000000,0,0,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,6.1
5769,1191380,00001000110110100010,0,0,0,0,1,0,0,0,...,1,1,0,1,0,0,0,1,0,6.9
5770,54540,10000000001000010010,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,7.1
5771,1673588,00000001010000000000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.8
